# Exploratory Analysis for TravelTide Marketing Project

## Goal is to segment customer behavior to target specific perks for the new rewards program to improve customer retention

## Perks
<ul>
    <li>Free Hotel Meal</li>
    <li>Free Checked Bag</li>
    <li>No Cancellation Fees</li>
    <li>Exclusive Discounts</li>
    <li>Exclusive Seasonal Discounts and Offers</li>
    <li>1 Night Free Hotel With Flight</li>
    <li>Complementary Lounge Access</li>
</ul>

Initial exploration was done in the Travel Tide Exploration Viz notebook but was starting to get sidetracked. I am moving forward with exploring potential segments in this notebook.